In [2]:
import pyarrow as pa
import pyarrow.json as paj
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pandas as pd
import json
import numpy as np
from IPython.core.display import display, HTML
from collections import defaultdict

json_file_path = r"C:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\Thesis_bestanden\__MACOSX\foodb_2020_04_07_json\Content.json"
parquet_file_path = json_file_path.replace(".json", ".parquet")
food_ids = [334, 483, 506, 549] #334 chicken, 483 mutton, 506 beef, 541 domesticated pig
columns_to_keep = [
    "id", "food_id", "orig_food_id", "orig_food_common_name",
    "source_id", "orig_source_id", "orig_source_name",
    "source_type", "orig_content", "orig_unit"
]

print("Converting JSON to Parquet (this may take a moment)...")
table = paj.read_json(json_file_path)
pq.write_table(table, parquet_file_path, compression="snappy")

food_ids_arrow = pa.array(food_ids, type=table.column("food_id").type)

print("Filtering data...")
table = pq.read_table(parquet_file_path)
filtered_table = table.filter(pc.is_in(table["food_id"], value_set=food_ids_arrow))
foodb_meat = filtered_table.select(columns_to_keep).to_pandas()

C:\Users\bramd_finhsgu\AppData\Local\Temp\ipykernel_19640\853975863.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


Converting JSON to Parquet (this may take a moment)...
Filtering data...


In [17]:
#1 filteren op orig_content
foodb_meat_filtered = foodb_meat.dropna(subset=["orig_content"]).reset_index(drop=True)
foodb_meat_filtered = foodb_meat_filtered.dropna(subset=["orig_food_common_name"]).reset_index(drop=True)

#2 filteren op food_common_name
unique_foods = foodb_meat_filtered.drop_duplicates(subset=["orig_food_common_name", "orig_food_id"])[["orig_food_common_name", "orig_food_id"]].values

#2.1 Set met vleesdelen/termen waarop gefilterd wordt
meat_parts = {"chuck", "rib", "short loin", "sirloin", "round", "brisket", "neck", "mince", "minced", "shoulder", "wing", "drumstick", 
              "tenderloin", "meat", "topside", "loin", "shank", "plate", "flank", "ribeye", "oxtail", "tongue", "leg", "belly", "hand",
              "rump", "entrecote", "fillet", "striploin", "t-bone", "strip steak", "breast", "cheek", "collar", "chop", "ham", "oyster"}
cooked_terms = {"cooked", "stewed", "boiled", "grilled", "roasted", "fried", "smoked", "braised", "cured", "roastbeef", "pastrami", "hormel", "mayer", 
                "canned", "prepackaged", "applebee's", "denny's", "friday's"}
organ_terms = {"brain", "kidney", "liver", "lungs", "heart", "pancreas", "thymus", "spleen", "suet", "tripe", "sweatbread"}

def filter_meat_parts(food_list, meat_parts, cooked_terms, organ_terms):
    """
    Filtert unieke vleesdelen en bijbehorende 'orig_food_id', en splitst de niet-gematchte items op in kooktermen of organen.

    Args:
        food_list (list): Lijst met unieke (food_name, food_id) paren.
        meat_parts (set): Set met relevante vleesstukken.
        cooked_terms (set): Set met woorden die op bereiding wijzen.
        organ_terms (set): Set met woorden die organen aanduiden.

    Returns:
        tuple: (meat_matches, non_meat_matches, organ_matches, cooked_matches)
    """
    meat_matches = {}  # Vleesdeel → food_id
    non_meat_matches = {}  # Niet-vlees of onbekend → food_id
    organ_matches = {}  # Organen → food_id
    cooked_matches = {}  # Vlees met kooktermen → food_id

    for food_name, food_id in food_list:
        if food_name is None:
            continue  # Sla over als de naam None is

        food_name_lower = food_name.lower()  # Zet food_name naar kleine letters voor hoofdletterongevoeligheid

        if any(term in food_name_lower for term in organ_terms):  # 1. Eerst organen filteren
            organ_matches[food_name] = food_id  # Opslaan als orgaan
        elif any(term in food_name_lower for term in cooked_terms):  # 2. Daarna koken/bereiding filteren
            cooked_matches[food_name] = food_id  # Opslaan als vlees met kookterm
        elif any(part in food_name_lower for part in meat_parts):  # 3. Daarna filteren op de opgegeven vleestermen
            meat_matches[food_name] = food_id  # Opslaan als uniek vleesdeel
        else:  # 4. Alles wat overblijft gaat naar non_meat_matches
            non_meat_matches[food_name] = food_id  # Opslaan als niet-vlees of onbekend

    return meat_matches, non_meat_matches, organ_matches, cooked_matches

meat_matches, non_meat_matches, organ_matches, cooked_matches = filter_meat_parts(unique_foods, meat_parts, cooked_terms, organ_terms)
meat_names = set(meat_matches.keys())  # Zet keys om in een set voor snelle lookup
foodb_meat_filtered_2 = foodb_meat_filtered[foodb_meat_filtered["orig_food_common_name"].isin(meat_names)]

#3 orig_source_name aanvullen op basis van source_id
def load_json_as_dict(json_path, key_field, value_field):
    """Laadt een NDJSON-bestand als een dictionary, waarbij elke regel een JSON-object is."""
    data_dict = {}
    with open(json_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                item = json.loads(line.strip())  # Elke regel apart parsen
                if key_field in item and value_field in item:
                    data_dict[str(item[key_field])] = item[value_field]
            except json.JSONDecodeError:
                print(f"Fout bij lezen van regel: {line}")  # Debugging
    return data_dict

nutrient_path = r"C:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\Thesis_bestanden\__MACOSX\foodb_2020_04_07_json\Nutrient.json"
compound_path = r"C:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\Thesis_bestanden\__MACOSX\foodb_2020_04_07_json\Compound.json"
nutrient_dict = load_json_as_dict(nutrient_path, 'id', 'name')
compound_dict = load_json_as_dict(compound_path, 'id', 'name')
compound_subklass_dict = load_json_as_dict(compound_path, 'id', 'subklass')
unmatched_counts = defaultdict(int)

def lookup_source_name(row):
    if pd.notna(row['orig_source_name']):  # Als al gevuld, overslaan
        return row['orig_source_name']
    
    source_id = str(row['source_id'])  # Omzetten naar string voor consistentie
    source_type = row['source_type']  # Controleren op onverwachte waarden

    if source_type == 'Nutrient':
        result = nutrient_dict.get(source_id, None)
    elif source_type == 'Compound':
        result = compound_dict.get(source_id, None)
    else:
        result = None  # Onverwacht type
    
    if result is None:
        unmatched_counts[(source_type, source_id)] += 1  # Aantal keer verhogen
    
    return result

def lookup_subklass(row):
    if row['source_type'] == 'Compound':
        return compound_subklass_dict.get(str(row['source_id']), None)  # Source ID als string voor consistentie
    return None

foodb = foodb_meat_filtered_2.copy()  # Maak een kopie om het origineel niet te overschrijven
foodb['orig_source_name'] = foodb.apply(lookup_source_name, axis=1)
foodb['subklass'] = foodb.apply(lookup_subklass, axis=1)

#3.1 Opslaan van niet-gematchte items als CSV voor manuele aanvulling van overblijvende orig_source_names waarvoor source_id geen koppeling bood.
unmatched_df = pd.DataFrame(unmatched_counts.items(), columns=['Source', 'Count'])
unmatched_df[['Source_Type', 'Source_ID']] = pd.DataFrame(unmatched_df['Source'].tolist(), index=unmatched_df.index)
unmatched_df = unmatched_df.drop(columns=['Source'])
unmatched_output_path = r"C:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\unmatched_sources.csv"
unmatched_df.to_csv(unmatched_output_path, index=False)
print(f"Niet-gematchte source_ids opgeslagen in: {unmatched_output_path}")


Niet-gematchte source_ids opgeslagen in: C:\Users\bramd_finhsgu\OneDrive - UGent\Thesis\unmatched_sources.csv


In [18]:
# Manueel toevoegen van overblijvende orig_source_names:
manual_names = {
    4858: '', #pagina 4, 8, ... van content bij food chicken, idem voor deze 2 hieronder
    13393: '',
    11820: '(R)-Pantothenic acid',
    21468: 'L-Dehydroascorbic acid',
    3519: 'Magnesium',
    12065: 'Butanoic acid',
    2890: 'Tetradecanoic acid',
    11682: 'Hexadecanoic acid',
    2942: 'Octadecanoic acid',
    2928: 'Eicosanoic acid',
    21973: 'Behenic acid',
    21981: 'Tetracosanoic acid',
    11678: 'Palmitoleic acid',
    3006: 'Cetoleic acid',
    6288: 'alpha-Linoleic acid',
    14708: 'L-Phenylalanine',
    465: 'L-Valine',
}

# Functie om de handmatig ingevulde naam te zoeken
def fill_manual_name(row):
    if pd.notna(row['orig_source_name']):
        return row['orig_source_name']  # Als er al een naam is, laat deze staan
    return manual_names.get(row['source_id'], None)  # Vul de naam in als die is ingevuld

# Pas de functie toe op de foodb_meat_filtered DataFrame
foodb['orig_source_name'] = foodb.apply(fill_manual_name, axis=1)
foodb_test['orig_source_name'] = foodb_test.apply(fill_manual_name, axis=1)

# Nu is foodb_meat_filtered bijgewerkt met de handmatig ingevulde namen
# Het resultaat is een DataFrame die je verder kunt gebruiken in je analyse
print("Het 'orig_source_name' veld is bijgewerkt in de DataFrame.")

Het 'orig_source_name' veld is bijgewerkt in de DataFrame.


In [19]:
counts_before = foodb_meat["food_id"].value_counts().sort_index()
counts_filter1 = foodb_meat_filtered["food_id"].value_counts().sort_index()
counts_filter2 = foodb_meat_filtered_2["food_id"].value_counts().sort_index()

print("\nAantal rijen per food_id vóór en na filtering:")
for food_id in sorted(set(counts_before.index) | set(counts_filter1.index) | set(counts_filter2.index)):  
    count_before = counts_before.get(food_id, 0)
    count_filter1 = counts_filter1.get(food_id, 0)
    count_filter2 = counts_filter2.get(food_id, 0)
    percentage_remaining = (count_filter2 / count_before * 100) if count_before > 0 else 0
    print(f"Food ID {food_id}: {count_before} rijen initieel → {count_filter1} rijen met orig_content waarde, {count_filter2}, met een food_name ({percentage_remaining:.2f}%)")

print(f"\nIn totaal waren er initieel {len(foodb_meat)} rijen, maar na filtering blijven er slechts {len(foodb_meat_filtered_2)} over. ({(len(foodb_meat_filtered_2) / len(foodb_meat) * 100):.2f}% resterend)\n")

def calculate_missing_matrix(df):
    missing_matrix = df.drop(columns=["food_id"]).groupby(df["food_id"]).apply(lambda x: x.isnull().mean() * 100).T
    missing_matrix = missing_matrix.round(2)

    # Bereken % HMDB aanwezigheid
    hmdb_percentage = df.groupby("food_id")["orig_food_common_name"].apply(
        lambda x: (x == "Endogenous compounds from human (HMDB)").mean() * 100
    )
    missing_matrix.loc["% HMDB aanwezigheid"] = hmdb_percentage.round(2)

    return missing_matrix

# Bereken de matrices
missing_matrix_initial = calculate_missing_matrix(foodb_meat)
missing_matrix_filter1 = calculate_missing_matrix(foodb_meat_filtered_2)
missing_matrix_filter2 = calculate_missing_matrix(foodb)

# Visuele separator
separator = pd.DataFrame("|", index=missing_matrix_initial.index, columns=["SEPARATOR"])

# Combineer de matrices zonder onnodige whitespace
comparison_matrix = pd.concat(
    [missing_matrix_initial, separator, missing_matrix_filter1, separator, missing_matrix_filter2], 
    axis=1)

# Zorg dat er geen extra whitespace komt na de eerste kolom
comparison_matrix.columns = pd.MultiIndex.from_tuples([
    ("Vóór filtering", col) if col != "SEPARATOR" else ("", col) for col in missing_matrix_initial.columns
] + [("", "SEPARATOR")] +
[
    ("Filtering 1", col) if col != "SEPARATOR" else ("", col) for col in missing_matrix_filter1.columns
] + [("", "SEPARATOR")] +
[
    ("Filtering 2", col) for col in missing_matrix_filter2.columns
])

# Verbeter de print-layout
pd.set_option("display.width", 300)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

print("\n=== Matrix van ontbrekende waarden en HMDB-aanwezigheid per food_id (%) ===\n")
print(comparison_matrix.to_string())



Aantal rijen per food_id vóór en na filtering:
Food ID 334: 51588 rijen initieel → 1510 rijen met orig_content waarde, 315, met een food_name (0.61%)
Food ID 483: 51102 rijen initieel → 945 rijen met orig_content waarde, 840, met een food_name (1.64%)
Food ID 506: 138620 rijen initieel → 88469 rijen met orig_content waarde, 33694, met een food_name (24.31%)
Food ID 549: 101966 rijen initieel → 52085 rijen met orig_content waarde, 19682, met een food_name (19.30%)

In totaal waren er initieel 343276 rijen, maar na filtering blijven er slechts 54531 over. (15.89% resterend)


=== Matrix van ontbrekende waarden en HMDB-aanwezigheid per food_id (%) ===

                      Vóór filtering                                 Filtering 1                                Filtering 2                      
                                 334     483    506    549 SEPARATOR         334    483    506    549 SEPARATOR         334     483    506    549
id                              0.00    0.00   0.

In [20]:
import json
import tempfile
import os

# Specificeer handmatig de food_id waarvoor je de data wilt bekijken
selected_food_id = 506  # <-- Pas deze waarde aan

# Filter de dataset op de geselecteerde food_id
filtered_df = foodb_meat[foodb_meat["food_id"] == selected_food_id]

# Converteer de gefilterde DataFrame naar JSON
json_data = filtered_df.to_dict(orient="records")
temp_json_path = os.path.join(tempfile.gettempdir(), "temp_json_view.json")

# Schrijf de JSON, met elk object op een aparte regel
with open(temp_json_path, "w", encoding="utf-8") as f:
    for record in json_data:
        json.dump(record, f, separators=(',', ':'), ensure_ascii=False)
        f.write("\n")  # Nieuwe regel voor elk object

# Open bestand in VS Code
os.system(f"code {temp_json_path}")

print(f"JSON-bestand geopend in VS Code voor food_id {selected_food_id}")


JSON-bestand geopend in VS Code voor food_id 506


In [21]:
# Veronderstel dat je dataframe 'foodb' heet
# Stap 1: Filter de rijen waar source_type gelijk is aan "compound"
foodb_filtered = foodb[foodb['source_type'] == 'Compound']

# Stap 2: Groepeer de data op 'orig_source_name' en 'food_id' en tel het aantal keer dat deze combinatie voorkomt
foodb_grouped = foodb_filtered.groupby(['orig_source_name', 'food_id']).size().reset_index(name='count')

# Stap 3: Zet de data om naar breed formaat (pivot tabel)
foodb_wide = foodb_grouped.pivot_table(index='orig_source_name', columns='food_id', values='count', fill_value=0)
foodb_wide = foodb_wide.astype(int)

# Stap 4: Voeg de source_id en subklass kolom toe via een merge (geen extra filtering)
source_info_mapping = foodb_filtered[['orig_source_name', 'source_id', 'subklass']].drop_duplicates()
foodb_wide = foodb_wide.reset_index().merge(source_info_mapping, on='orig_source_name', how='left')

# Stap 5: Zet de juiste index en kolomvolgorde
foodb_wide = foodb_wide.set_index(['orig_source_name', 'source_id', 'subklass'])

# Bekijk de output
total_unique_compounds = foodb_wide.shape[0]  # Aantal unieke orig_source_names
unique_compounds_per_food_id = (foodb_wide > 0).sum(axis=0)  # Aantal unieke compounds per food_id

# Unieke source_id's tellen
unique_source_ids = foodb_filtered['source_id'].nunique()

# Unieke source_id's per food_id tellen
unique_source_ids_per_food_id = foodb_filtered.groupby('food_id')['source_id'].nunique()

# Tekstoutput genereren
summary_text = f"Totaal aantal unieke compounds: {total_unique_compounds}\n"
summary_text += "Aantal unieke compounds per food_id:\n"
for food_id, count in unique_compounds_per_food_id.items():
    summary_text += f"- Food ID {food_id}: {count}\n"

summary_text += f"\nTotaal aantal unieke source_id's: {unique_source_ids}\n"
summary_text += "Aantal unieke source_id's per food_id:\n"
for food_id, count in unique_source_ids_per_food_id.items():
    summary_text += f"- Food ID {food_id}: {count}\n"

pd.set_option('display.max_rows', None)  # Toon alle rijen
pd.set_option('display.max_columns', None)  # Toon alle kolommen
pd.set_option('display.width', None)  # Zorg dat de hele breedte wordt gebruikt
pd.set_option('display.expand_frame_repr', False)  # Vermijd dat Pandas regels splitst over meerdere lijnen
# Print de samenvatting
print(summary_text)
print(foodb_wide)


Totaal aantal unieke compounds: 239
Aantal unieke compounds per food_id:
- Food ID 334: 87
- Food ID 483: 87
- Food ID 506: 238
- Food ID 549: 236

Totaal aantal unieke source_id's: 127
Aantal unieke source_id's per food_id:
- Food ID 334: 87
- Food ID 483: 87
- Food ID 506: 127
- Food ID 549: 125

                                                                                                   334  483  506  549
orig_source_name                               source_id subklass                                                    
                                               4858      NaN                                         6   16   68  120
                                               13393     NaN                                         6   16   68  120
(E)-2-Phenyl-2-butenal                         16357     Phenylacetaldehydes                         3    8   34   60
(R)-Oxypeucedanin                              2520      NaN                                         3    8   

In [22]:
foodb.to_csv("foodb_data.csv", index=False, encoding="utf-8")